In [1]:
import optuna
from ultralytics import YOLO
import logging
import torch
import gc
import sys
import yaml
from pathlib import Path

ROOT_DIR = Path().resolve().parents[1]
sys.path.append(str(ROOT_DIR))
from src.utils.config_loader import load_config
cfg = load_config()

c:\Bureau\Proyectos\DATAGIA\Modelo-CU11\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Definición variables

Selección del modelo

In [ ]:
version_yolo = "experiment_yolo8s"

Asignación directorios

In [ ]:
model_yolo = ROOT_DIR / cfg["yolo_train"] / version_yolo
dataset_yaml = ROOT_DIR / cfg["dataset"]["tuning_yaml"] / "dataset.yaml"
dir_tun = ROOT_DIR / cfg["paths"]["model_tuning_path"]

# Creación función objective()

In [9]:
def objective(trial):
    # 1. Limpieza de VRAM (Vital para evitar el error anterior)
    gc.collect()
    torch.cuda.empty_cache()

    # 2. Espacio de búsqueda Bayesiano
    # Sugerimos rangos basados en el impacto detectado en clases minoritarias
    lr0 = trial.suggest_float("lr0", 1e-5, 1e-2, log=True)
    momentum = trial.suggest_float("momentum", 0.85, 0.95)
    mosaic = trial.suggest_float("mosaic", 0.0, 1.0) # Para desbalanceo
    mixup = trial.suggest_float("mixup", 0.0, 0.15)

    # 3. Configuración del entrenamiento
    model = YOLO(model_yolo / "weights/best.pt")

    results = model.train(
        data=dataset_yaml,
        project=dir_tun,
            name=f"trial_{trial.number}",
            exist_ok=True,
            save_period=5,    # Guarda cada 5 épocas para análisis posterior
            save=True,       # Guarda el mejor modelo de cada trial
            verbose=False,   # Reduce salida por consola para claridad
        epochs=15,         # Épocas suficientes para ver la tendencia de mAP
        batch=8,           # Batch reducido por estabilidad de VRAM
        imgsz=640,
        lr0=lr0,
        momentum=momentum,
        mosaic=mosaic,
        mixup=mixup,
        workers=0,         # Evita colisiones de hilos en Windows
        device=0,
        plots=False        # No genera imágenes en cada trial para ahorrar espacio
    )

    # 4. Retorno de la métrica a optimizar
    return results.results_dict['metrics/mAP50(B)']

# Bucle principal

In [ ]:
# Crear el estudio con un Pruner (poda) para mayor eficiencia energética
study = optuna.create_study(
    direction="maximize", 
    sampler=optuna.samplers.TPESampler(), # Algoritmo Bayesiano TPE
    pruner=optuna.pruners.MedianPruner()
)

study.optimize(objective, n_trials=20)

# Resultados para la Memoria Técnica
print(f"Mejor mAP50 obtenido: {study.best_value}")
print(f"Configuración ganadora: {study.best_params}")

[I 2026-02-17 12:34:07,074] A new study created in memory with name: no-name-8912a0e9-8cfc-4734-a926-390578f4e349


New https://pypi.org/project/ultralytics/8.4.14 available  Update with 'pip install -U ultralytics'
Ultralytics 8.4.11  Python-3.12.3 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4090, 24564MiB)
engine\trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\Bureau\Proyectos\DATAGIA\Modelo-CU11\notebooks\dataset.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=15, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0011653340652819353, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0530741256003274, mode=train, model=C:\B

[W 2026-02-17 12:34:32,509] Trial 0 failed with parameters: {'lr0': 0.0011653340652819353, 'momentum': 0.8914431722041776, 'mosaic': 0.6610104088613035, 'mixup': 0.0530741256003274} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Bureau\Proyectos\DATAGIA\Modelo-CU11\env\Lib\site-packages\optuna\study\_optimize.py", line 206, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\USUARIO\AppData\Local\Temp\ipykernel_37088\2559805297.py", line 16, in objective
    results = model.train(
              ^^^^^^^^^^^^
  File "c:\Bureau\Proyectos\DATAGIA\Modelo-CU11\env\Lib\site-packages\ultralytics\engine\model.py", line 774, in train
    self.trainer.train()
  File "c:\Bureau\Proyectos\DATAGIA\Modelo-CU11\env\Lib\site-packages\ultralytics\engine\trainer.py", line 244, in train
    self._do_train()
  File "c:\Bureau\Proyectos\DATAGIA\Modelo-CU11\env\Lib\site-packages\ultralytics\engine\trainer.py", 

KeyboardInterrupt: 